In [1]:
# Please install libjpeg-turbo and then jpeg4py 
# https://sourceforge.net/projects/libjpeg-turbo/
# pip install jpeg4py
# then install prefetch_generator by using pip: pip install prefetch_generator
# before training please change the HOME directory location in the config file present at 'data/config.py'

In [2]:
import sys
print(sys.executable)

C:\Users\Anubhav\anaconda3\envs\mypytorch\python.exe


In [3]:
!conda env list

# conda environments:
#
base                     C:\Users\Anubhav\anaconda3
acii                     C:\Users\Anubhav\anaconda3\envs\acii
amigos                   C:\Users\Anubhav\anaconda3\envs\amigos
ideas                    C:\Users\Anubhav\anaconda3\envs\ideas
mypytorch             *  C:\Users\Anubhav\anaconda3\envs\mypytorch
neuralcar                C:\Users\Anubhav\anaconda3\envs\neuralcar
projectx                 C:\Users\Anubhav\anaconda3\envs\projectx
puzzle                   C:\Users\Anubhav\anaconda3\envs\puzzle



In [4]:
import jpeg4py as jpeg
from PIL import Image

# # img = Image.open(r'C:\Users\Anubhav\data\VOCdevkit\VOC2007\JPEGImages\000001.jpg')
# img = jpeg.JPEG(source=r'C:\Users\Anubhav\data\VOCdevkit\VOC2007\JPEGImages\000001.jpg').decode()

In [5]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [6]:
torch.cuda.is_available()

True

In [7]:
import sys
print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [8]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd_mobilenet_se import build_ssd_mobilenet_se
import os
import sys
import time
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import numpy as np
from prefetch_generator import BackgroundGenerator
import datetime

In [9]:
from torch.utils.data import DataLoader
class DataLoaderX(data.DataLoader):
    def __iter__(self):
        return BackgroundGenerator(super().__iter__())

In [10]:
args_warmup = False
args_backbone = 'mobilenet'
args_from_scratch = True
args_cuda = True
args_save_folder = 'weights/'
args_gamma = 0.1
args_weight_decay = 5e-4
args_momentum = 0.9
args_lr = 1e-3
args_num_workers = 2 # try -1
args_start_iter = 0
args_batch_size = 32
args_dataset_root = VOC_ROOT
args_dataset = 'VOC'

In [11]:
VOC_ROOT # contains the datasets

'C:/Users/Anubhav/Jupyter Projects/computer_vision [mypytorch]/ssd_mobv2/data/VOCdevkit/'

In [12]:
if torch.cuda.is_available():
    if args_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    if not args_cuda:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

if not os.path.exists(args_save_folder):
    os.mkdir(args_save_folder)

In [13]:
HOME

'C:/Users/Anubhav/Jupyter Projects/computer_vision [mypytorch]/ssd_mobv2/'

In [14]:
def warmup_learning_rate(optimizer, steps, warmup_steps):
    min_lr = args_lr / 100
    slope = (args_lr - min_lr) / warmup_steps

    lr = steps * slope + min_lr
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = args_lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def xavier(param):
    init.xavier_uniform(param)

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        if m.bias is not None:
            m.bias.data.zero_()

In [15]:
''' config files contains  HOME path, COLORS, MEANS, voc config'''
cfg = voc
dataset = VOCDetection(root=args_dataset_root, transform=SSDAugmentation(cfg['min_dim'], MEANS))

ssd_net = build_ssd_mobilenet_se('train', cfg)

print('Initializing weights...')
# initialize newly added layers' weights with xavier method
ssd_net.extras.apply(weights_init)
ssd_net.loc.apply(weights_init)
ssd_net.conf.apply(weights_init)

net = ssd_net
print(net)
print('extras:', net.extras)

if args_cuda:
    net = torch.nn.DataParallel(ssd_net)
    cudnn.benchmark = True

if args_cuda:
    net = net.cuda()

optimizer = optim.SGD(net.parameters(), lr=args_lr, momentum=args_momentum,
                        weight_decay=args_weight_decay)
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, 100)
criterion = MultiBoxLoss(cfg['num_classes'], 0.4, True, 0, True, 3, 0.5,
                            False, args_cuda)

net.train()
# loss counters
loc_loss = 0
conf_loss = 0
epoch = 0

print('Loading the dataset...')

epoch_size = len(dataset) // args_batch_size
print('Training SSD on:', args_dataset)

step_index = 0

C:/Users/Anubhav/Jupyter Projects/computer_vision [mypytorch]/ssd_mobv2/
C:/Users/Anubhav/Jupyter Projects/computer_vision [mypytorch]/ssd_mobv2/
Initializing weights...
SSDMobileNetSEV2(
  (backbone): ModuleList(
    (0): ConvBNReLU(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): CELU(alpha=1.0, inplace=True)
    )
    (1): InvertedResidual_SE(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): CELU(alpha=1.0, inplace=True)
        )
        (1): ConvBNReLU(
          (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=24, bias=False)
          (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [16]:
data_loader = DataLoaderX(dataset, args_batch_size,
                                num_workers=args_num_workers,
                                shuffle=True, collate_fn=detection_collate,
                                pin_memory=True)
# create batch iterator
batch_iterator = iter(data_loader)
t0 = time.time()

In [17]:
hist_loss = []
for iteration in range(args_start_iter, cfg['max_iter']):
    running_loss = 0.0

    if iteration != 0 and (iteration % epoch_size == 0):
        # reset epoch loss counters
        hist_loss.append(running_loss)
        loc_loss = 0
        conf_loss = 0
        epoch += 1

    if args_warmup:
        warmup_steps = 2000
        if iteration < warmup_steps:
            warmup_learning_rate(optimizer, iteration, warmup_steps)

    cosine_period = int(cfg['lr_steps'][-1] / 100)
    
    if iteration > cfg['lr_steps'][0] and iteration % cosine_period == 0:
        scheduler.step(iteration / cosine_period)

    # load train data
    try:
        images, targets = next(batch_iterator)
        # images, targets = data_loader
    except StopIteration:
        batch_iterator = iter(data_loader)
        images, targets = next(batch_iterator)
    except Exception as e:
        print("Loading data exception:", e)
        print('ISSUEE :(')

    with torch.no_grad():
        if args_cuda:
            images = Variable(images.cuda())
            targets = [Variable(ann.cuda()) for ann in targets]
        else:
            images = Variable(images)
            targets = [Variable(ann) for ann in targets]

    # forward
    out = net(images)
    # backprop
    optimizer.zero_grad()
    loss_l, loss_c = criterion(out, targets)
    loss = loss_l + loss_c
    loss.backward()

    nn.utils.clip_grad_norm_(net.parameters(), max_norm=20, norm_type=2)
    optimizer.step()
    loc_loss += loss_l.item()
    conf_loss += loss_c.item()

    running_loss += loss.item() * images.size(0)
    
    
    verbose_period = 1000
    if iteration != 0 and iteration % verbose_period == 0:
        print('timer: %.4f sec.' % (time.time() - t0))
        t0 = time.time()
        lr = optimizer.param_groups[0]['lr']
        print('iter ' + repr(iteration) + ' || Loc loss: %.4f || Conf loss: %.4f || lr: %.6f' % (loc_loss / verbose_period, conf_loss / verbose_period, lr), end=' ')
        # loc_loss = 0
        # conf_loss = 0

    if iteration != 0 and iteration % 5000 == 0:
        print('Saving state, iter:', iteration)
        torch.save(net, 'weights/ssdSE300_VOC_' + repr(iteration) + "__" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.pth')
torch.save(net, args_save_folder + '' + args_dataset + "_SE_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.pth')

C:\Users\Anubhav\anaconda3\envs\mypytorch\lib\site-packages\torch\nn\functional.py:2952: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
C:\Users\Anubhav\anaconda3\envs\mypytorch\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
timer: 436.5084 sec.
iter 1000 || Loc loss: 0.0447 || Conf loss: 0.0353 || lr: 0.001000 timer: 416.2601 sec.
iter 2000 || Loc loss: 0.0748 || Conf loss: 0.0663 || lr: 0.001000 timer: 413.6548 sec.
iter 3000 || Loc loss: 0.0221 || Conf loss: 0.0203 || lr: 0.001000 timer: 417.4506 sec.
iter 4000 || Loc loss: 0.0513 || Conf loss: 0.0512 || lr: 0.001000 timer: 428.5785 sec.
iter 5000 || Loc loss: 0.0067 || Conf loss: 0.0066 || lr: 0.001000 Saving state, iter: 5000
timer: 413.9170 sec.
iter 6000 || Loc 

In [17]:
args_save_folder

'weights/'

In [18]:
from matplotlib import pyplot as plt
import seaborn as sns

fig, ax = plt.subplots()

sns.lineplot(range(len(hist_loss)), hist_loss, ci=None)
plt.show()

In [21]:
sns.lineplot(range(len(hist_loss)), hist_loss, ci=None)

C:\Users\Anubhav\anaconda3\envs\mypytorch\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


In [20]:
import seaborn as sns

In [21]:
from matplotlib import pyplot as plt

plt.plot(hist_loss)

In [20]:
import pickle
with open('history_loss.pickle', 'wb') as f:
        pickle.dump(hist_loss, f, pickle.HIGHEST_PROTOCOL)